In [1]:
#r "nuget: Microsoft.Extensions.AI.Abstractions, 9.9.0"

Installed Packages Microsoft.Extensions.AI.Abstractions, 9.9.0

In [2]:
#r "nuget: Azure.AI.Agents.Persistent, 1.2.0-beta.4"
#r "nuget: Azure.Identity, 1.15.0"
#r "nuget: System.Linq.Async, 6.0.3"
#r "nuget: Microsoft.Extensions.AI, 9.8.0"
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages Azure.AI.Agents.Persistent, 1.2.0-beta.4 Azure.Identity, 1.15.0 DotNetEnv, 3.1.1 Microsoft.Extensions.AI, 9.8.0 System.Linq.Async, 6.0.3

In [3]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.Workflows\bin\Debug\net9.0\Microsoft.Agents.Workflows.dll"

In [4]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI.OpenAI\bin\Debug\net9.0/Microsoft.Agents.AI.OpenAI.dll"

In [5]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI\bin\Debug\net9.0\Microsoft.Agents.AI.dll"

In [6]:
#r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

Installed Packages Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4

In [7]:
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [8]:
using System;
using System.ClientModel;
using System.Text.Json;
using System.Text.Json.Serialization;
using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using Microsoft.Agents.Workflows;
using OpenAI;

In [9]:
 using DotNetEnv;

In [10]:
Env.Load("../../../.env");

In [11]:
var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

In [12]:
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint= new Uri(github_endpoint)
};

In [13]:
var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [14]:
const string REVIEWER_NAME = "Concierge";
const string REVIEWER_INSTRUCTIONS = @"""
    You are an are hotel concierge who has opinions about providing the most local and authentic experiences for travelers.
    The goal is to determine if the front desk travel agent has recommended the best non-touristy experience for a traveler.
    If so, state that it is approved.
    If not, provide insight on how to refine the recommendation without using a specific example. 
    """;

In [15]:
const string FRONTDESK_NAME = "FrontDesk";
const string FRONTDESK_INSTRUCTIONS = @"""
    You are a Front Desk Travel Agent with ten years of experience and are known for brevity as you deal with many customers.
    The goal is to provide the best activities and locations for a traveler to visit.
    Only provide a single recommendation per response.
    You're laser focused on the goal at hand.
    Don't waste time with chit chat.
    Consider suggestions when refining an idea.
    """;

In [16]:

ChatClientAgentOptions frontdeskAgentOptions = new(name: FRONTDESK_NAME, instructions: FRONTDESK_INSTRUCTIONS);
ChatClientAgentOptions reviewerAgentOptions = new(name: REVIEWER_NAME, instructions: REVIEWER_INSTRUCTIONS);

In [17]:
AIAgent reviewerAgent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).CreateAIAgent(
    reviewerAgentOptions);
AIAgent frontdeskAgent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).CreateAIAgent(
    frontdeskAgentOptions);

In [18]:
var workflow = new WorkflowBuilder(frontdeskAgent)
            .AddEdge(frontdeskAgent, reviewerAgent)
            .Build();

In [19]:
StreamingRun run = await InProcessExecution.StreamAsync(workflow, new ChatMessage(ChatRole.User, "I would like to go to Paris."));

In [20]:
await run.TrySendMessageAsync(new TurnToken(emitEvents: true));

In [21]:
string strResult = "";

In [22]:
await foreach (WorkflowEvent evt in run.WatchStreamAsync().ConfigureAwait(false))
{
            if (evt is AgentRunUpdateEvent executorComplete)
            {
                strResult += executorComplete.Data;
                Console.WriteLine($"{executorComplete.ExecutorId}: {executorComplete.Data}");
            }
}

983f518cfa884372b7c7bb656674703b: 
983f518cfa884372b7c7bb656674703b: Visit
983f518cfa884372b7c7bb656674703b:  the
983f518cfa884372b7c7bb656674703b:  Eiffel
983f518cfa884372b7c7bb656674703b:  Tower
983f518cfa884372b7c7bb656674703b:  for
983f518cfa884372b7c7bb656674703b:  breathtaking
983f518cfa884372b7c7bb656674703b:  views
983f518cfa884372b7c7bb656674703b:  and
983f518cfa884372b7c7bb656674703b:  a
983f518cfa884372b7c7bb656674703b:  quintessential
983f518cfa884372b7c7bb656674703b:  Paris
983f518cfa884372b7c7bb656674703b: ian
983f518cfa884372b7c7bb656674703b:  experience
983f518cfa884372b7c7bb656674703b: .
983f518cfa884372b7c7bb656674703b: 
983f518cfa884372b7c7bb656674703b: 
1b5064ffbcfd4036a206b293a304eac5: 
1b5064ffbcfd4036a206b293a304eac5: This
1b5064ffbcfd4036a206b293a304eac5:  recommendation
1b5064ffbcfd4036a206b293a304eac5:  is
1b5064ffbcfd4036a206b293a304eac5:  not
1b5064ffbcfd4036a206b293a304eac5:  approved
1b5064ffbcfd4036a206b293a304eac5: .
1b5064ffbcfd4036a206b293a304eac5:  Wh

In [23]:
strResult

Visit the Eiffel Tower for breathtaking views and a quintessential Parisian experience.This recommendation is not approved. While the Eiffel Tower is iconic, it is highly touristic and can overshadow more authentic local experiences. I suggest focusing on smaller, lesser-known attractions or local spots where travelers can immerse themselves in the culture, perhaps by exploring neighborhoods or engaging in community events. This will provide a more genuine taste of the city's charm.